In [102]:
import pandas as pd

In [103]:
# Read original csv files and drop unnecessary columns

aids_deaths = pd.read_csv("../Dataset/raw_data/aids_deaths.csv", sep=";")
countries_and_populations = pd.read_csv("../Dataset/raw_data/countries_and_populations.csv", sep=";")
death_averted_art = pd.read_csv("../Dataset/raw_data/death_averted_art.csv", sep=";")
hiv_death_rates = pd.read_csv("../Dataset/raw_data/hiv_death_rates.csv", sep=";")
human_dev_index = pd.read_csv("../Dataset/raw_data/human_dev_index.csv", sep=";")
life_expectancy = pd.read_csv("../Dataset/raw_data/life_expectancy.csv", sep=";")
num_deaths = pd.read_csv("../Dataset/raw_data/num_deaths.csv", sep=";")
country_continent = pd.read_csv("../Dataset/raw_data/country_continent.csv", sep=";")

country_table = countries_and_populations
country_table.drop(columns="Year", inplace=True)

country_continent.rename(columns={"code_3": "Code"}, inplace=True)

In [104]:
# Life expectancy raw table:

life_expectancy_table = life_expectancy.drop(columns=["Year"])
life_expectancy_table = life_expectancy_table.rename(columns={"Entity": "Country"})
life_expectancy_table.dropna(inplace=True)
life_expectancy_table

,Country,Code,Life expectancy at birth (historical)
0,Afghanistan,AFG,"63,6"
2,Albania,ALB,"79,3"
3,Algeria,DZA,"76,5"
4,American Samoa,ASM,"72,3"
6,Andorra,AND,83
...,...,...,...
249,Western Sahara,ESH,"70,3"
250,World,OWID_WRL,"72,8"
251,Yemen,YEM,"65,1"
252,Zambia,ZMB,"62,8"


In [105]:
# Country table:
country_merged = (country_table.merge(life_expectancy_table, how="inner", on="Country")).drop(columns=["Life expectancy at birth (historical)"])


# Drop countries that do not exist in country table from life expectancy table:
life_expectancy_table_merged = life_expectancy_table.merge(country_merged, how="inner", on="Code")
life_expectancy_table_merged.drop(columns=["Country_x", "Country_y", "Population"], inplace=True)
life_expectancy_table_merged

# Finalize country table:
country_merged = (country_merged.merge(human_dev_index.drop(columns=["Country"]), how="left", on="Code")).drop(columns=["Year"])
country_merged = country_merged.merge(country_continent.drop(columns="country"), how="left", on="Code")
country_merged

# 181 Countries!

,Country,Population,Code,Human Development Index,continent
0,Afghanistan,37769496,AFG,"0,488",Asia
1,Albania,2873883,ALB,"0,81",Europe
2,Algeria,42705372,DZA,"0,748",Africa
3,Andorra,76361,AND,"0,873",Europe
4,Angola,32353592,AGO,"0,595",Africa
...,...,...,...,...,...
176,Vietnam,95776710,VNM,"0,703",Asia
177,World,7764951000,OWID_WRL,"0,739",NaN
178,Yemen,31546694,YEM,"0,461",Asia
179,Zambia,18380478,ZMB,"0,575",Africa


In [106]:
# Hiv death rates table: 
hiv_deaths_table = hiv_death_rates.drop(columns="Year")
hiv_deaths_table["death_rate_age_5-"] = hiv_death_rates["Deaths - HIV/AIDS - Sex: Both - Age: Under 5 (Rate)"].copy()
hiv_deaths_table["death_rate_age_5-14"] = hiv_death_rates["Deaths - HIV/AIDS - Sex: Both - Age: 5-14 years (Rate)"].copy()
hiv_deaths_table["death_rate_age_14-49"] = hiv_death_rates["Deaths - HIV/AIDS - Sex: Both - Age: 15-49 years (Rate)"].copy()
hiv_deaths_table["death_rate_age_50-69"] = hiv_death_rates["Deaths - HIV/AIDS - Sex: Both - Age: 50-69 years (Rate)"].copy()
hiv_deaths_table["death_rate_age_70+"] = hiv_death_rates["Deaths - HIV/AIDS - Sex: Both - Age: 70+ years (Rate)"].copy()

hiv_deaths_table_dropped = hiv_deaths_table.drop(columns=["Deaths - HIV/AIDS - Sex: Both - Age: Under 5 (Rate)", "Deaths - HIV/AIDS - Sex: Both - Age: 5-14 years (Rate)", "Deaths - HIV/AIDS - Sex: Both - Age: 15-49 years (Rate)", "Deaths - HIV/AIDS - Sex: Both - Age: 50-69 years (Rate)", "Deaths - HIV/AIDS - Sex: Both - Age: All Ages (Rate)", "Deaths - HIV/AIDS - Sex: Both - Age: 70+ years (Rate)", "Entity"])
hiv_deaths_table_dropped = hiv_deaths_table_dropped.dropna()
hiv_deaths_table_dropped

,Code,death_rate_age_5-,death_rate_age_5-14,death_rate_age_14-49,death_rate_age_50-69,death_rate_age_70+
0,AFG,"1,48","0,05","0,99","1,36","0,6"
2,ALB,"0,06","0,01","0,07","0,08","0,03"
3,DZA,"0,43","0,05","0,69","1,19","0,85"
4,ASM,"0,71","0,15","1,13","1,13","0,63"
5,AND,"0,48","0,02","3,44","5,69","4,43"
...,...,...,...,...,...,...
217,VNM,"0,07","0,01","11,1","2,78","0,16"
220,OWID_WRL,"7,38","2,12","14,94","13,23","3,66"
225,YEM,"1,94","0,09","1,33","1,7","0,84"
226,ZMB,"107,18","19,58","157,67","333,02","116,9"


In [107]:
# Deaths table:
deaths_table = num_deaths.copy()[0:0].drop(columns=["Deaths - Meningitis - Sex: Both - Age: All Ages (Number)", "Deaths - HIV/AIDS - Sex: Both - Age: All Ages (Number)", "Deaths - Tuberculosis - Sex: Both - Age: All Ages (Number)", "Deaths - Cardiovascular diseases - Sex: Both - Age: All Ages (Number)", "Deaths - Acute hepatitis - Sex: Both - Age: All Ages (Number)", "Entity"])
deaths_table["Num Deaths"] = deaths_table["Disease Type"] = deaths_table["Disase Code"] = []

i = 0
for row in num_deaths.itertuples():
    deaths_table.loc[i] = [row[2], row[3], row[4], "meningitis", 0]
    deaths_table.loc[i+1] = [row[2], row[3], row[5], "HIV/AIDS", 1]
    deaths_table.loc[i+2] = [row[2], row[3], row[6], "Tuberculosis", 2]
    deaths_table.loc[i+3] = [row[2], row[3], row[7], "cardiovascular", 3] 
    i += 4

deaths_table.dropna(inplace=True)
deaths_table


,Code,Year,Num Deaths,Disease Type,Disase Code
0,AFG,2019,1563,meningitis,0
1,AFG,2019,318,HIV/AIDS,1
2,AFG,2019,3627,Tuberculosis,2
3,AFG,2019,61995,cardiovascular,3
8,ALB,2019,13,meningitis,0
...,...,...,...,...,...
907,ZMB,2019,18157,cardiovascular,3
908,ZWE,2019,1450,meningitis,0
909,ZWE,2019,20722,HIV/AIDS,1
910,ZWE,2019,10465,Tuberculosis,2


In [108]:
# Cures by art table:
cures_by_art_table = death_averted_art.drop(columns=["Entity", "Unnamed: 4"])
cures_by_art_table

,Code,Year,Deaths averted due to ART - estimate
0,AFG,2016,"41,4994"
1,ALB,2016,"44,8595"
2,DZA,2016,"483,807"
3,AGO,2016,"6064,21"
4,ARG,2016,"2708,42"
...,...,...,...
130,NaN,2016,"59664,7"
131,OWID_WRL,2016,1210000
132,YEM,2016,"139,635"
133,ZMB,2016,"91546,8"


In [109]:
# Cause table:
cause_table = deaths_table.copy()
cause_table = cause_table.drop(columns=["Year", "Num Deaths", "Disease Type"])
cause_table = cause_table.merge(country_merged.drop(columns=["Country", "Population", "Human Development Index", "continent"]), how="right", on="Code")
cause_table

,Code,Disase Code
0,AFG,0.0
1,AFG,1.0
2,AFG,2.0
3,AFG,3.0
4,ALB,0.0
...,...,...
677,ZMB,3.0
678,ZWE,0.0
679,ZWE,1.0
680,ZWE,2.0


In [131]:
# Finalize column names:
country_final = country_merged.rename(columns={"Country": "name", "Population": "population", "Code": "code", "Human Development Index": "development_index"})
life_expectancy_final = life_expectancy_table_merged.rename(columns={"Country": "country_name", "Code": "code", "Life expectancy at birth (historical)": "age"})
expect_final = life_expectancy_final[["code"]].copy()
cures_by_art_final = cures_by_art_table.drop(columns="Year").rename(columns={"Deaths averted due to ART - estimate": "num_deaths_averted"})
hiv_deaths_final = hiv_deaths_table_dropped.rename(columns={"Code": "country_code"})
deaths_final = deaths_table.rename(columns={"Code": "country_code", "Year": "date", "Num Deaths": "death_num", "disease_type": "is_a", "Disase Code": "disease_code"})
cause_final = cause_table.rename(columns={"Code": "country_code", "Disase Code": "disease_code"})

In [133]:
# Output the resulting csv's:
country_final.to_csv("../Dataset/organized_data/country.csv", index=False)
life_expectancy_final.to_csv("../Dataset/organized_data/life_expectancy.csv", index=False)
expect_final.to_csv("../Dataset/organized_data/expect.csv", index=False)
cures_by_art_final.to_csv("../Dataset/organized_data/cures_by_art.csv", index=False)
hiv_deaths_final.to_csv("../Dataset/organized_data/hiv_deaths.csv", index=False)
deaths_final.to_csv("../Dataset/organized_data/deaths.csv", index=False)
cause_final.to_csv("../Dataset/organized_data/cause.csv", index=False)